In [1]:
from apscheduler.schedulers.asyncio import AsyncIOScheduler
from telegram import Bot
from telegram.ext import Application, CommandHandler
import requests
import feedparser
import re
from datetime import datetime
import pytz
import asyncio

In [2]:
# Список городов с их кодами для RSS-ленты
CITIES = {
    "Москва (ВДНХ)": "27612",
    "Москва (Шереметьево)": "27514",
    "Белый": "27613",
    "Смоленск": "26781",
    "Великие Луки": "26477",
    "Торопец": "26479",
    "Тверь": "27402",
    "Старица": "26499",
}

In [3]:
# Словарь для хранения последнего состояния прогноза для каждого города
last_weather_data = {}

In [4]:
# Функция для получения прогноза погоды из RSS
def get_weather_from_rss(city_name, city_code):
    rss_url = f"https://meteoinfo.ru/rss/forecasts/index.php?s={city_code}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
    
    try:
        response = requests.get(rss_url, headers=headers)
        response.raise_for_status()  # Проверка на HTTP-ошибки
        
        # Принудительно парсим содержимое как XML
        feed = feedparser.parse(response.content)
        
        if feed.bozo:  # Проверка на ошибки при парсинге
            return f"Ошибка при парсинге RSS для города {city_name}: {feed.bozo_exception}"
        
        # Извлекаем время обновления прогноза из тега <source>
        update_time_formatted = None
        for entry in feed.entries:
            source_title = ""
            if "source" in entry and isinstance(entry["source"], dict):
                source_title = entry["source"].get("title", "").strip()
            
            # Ищем строку с временем обновления
            if source_title:
                update_time_match = re.search(
                    r"прогноз\s+обновл[её]н\s+(\d{2}\.\d{2}\.\d{4} в \d{2}:\d{2})\(utc\)", 
                    source_title, 
                    re.IGNORECASE
                )
                if update_time_match:
                    # Преобразуем время в объект datetime
                    update_time_str = update_time_match.group(1)
                    update_time_utc = datetime.strptime(update_time_str, "%d.%m.%Y в %H:%M")
                    
                    # Указываем, что это время в UTC
                    utc_zone = pytz.utc
                    update_time_utc = utc_zone.localize(update_time_utc)
                    
                    # Конвертируем в локальное время
                    local_zone = pytz.timezone("Europe/Moscow")  # Замените на ваш часовой пояс
                    update_time_local = update_time_utc.astimezone(local_zone)
                    
                    # Форматируем время в читаемый вид
                    update_time_formatted = update_time_local.strftime("%d.%m.%Y в %H:%M")
        
        # Название города из первого элемента
        weather_data = [f"*Прогноз погоды от Гидрометцентра России* 🌡️", f"🏠 *{city_name}*"]
        
        parsed_data = []
        for entry in feed.entries:
            title = entry.get("title", "Заголовок отсутствует")
            description = entry.get("summary", "Описание отсутствует")  # Используем summary вместо description
            
            # Извлекаем только дату из заголовка
            date_match = re.search(r"\d{1,2} [а-я]+", title)
            date = date_match.group(0) if date_match else "Дата отсутствует"
            
            # Парсим описание погоды
            weather_description = description.split(". ")[0]
            
            # Парсим температуру
            temperature_match = re.search(r"Температура ночью (\d+)°, днём (\d+)°", description)
            night_temp = int(temperature_match.group(1)) if temperature_match else None
            day_temp = int(temperature_match.group(2)) if temperature_match else None
            
            # Парсим ветер
            wind_match = re.search(r"Ветер ([а-яё\-]+), (\d+) м/с", description)
            wind_direction = wind_match.group(1) if wind_match else None
            wind_speed = int(wind_match.group(2)) if wind_match else None
            
            # Парсим давление
            pressure_match = re.search(r"Атмосферное давление ночью (\d+) мм рт\.ст\., днём (\d+) мм рт\.ст", description)
            night_pressure = int(pressure_match.group(1)) if pressure_match else None
            day_pressure = int(pressure_match.group(2)) if pressure_match else None
            
            # Парсим вероятность осадков
            precipitation_match = re.search(r"Вероятность осадков (\d+%)", description)
            precipitation = int(precipitation_match.group(1).rstrip("%")) if precipitation_match else None
            
            # Формируем сообщение с иконками
            weather_message = (
                f"🗓️ *{date}*\n"
                f"*{weather_description}* 🔹\n"  # Описание погоды жирным
                f"🌡️ Температура ночью {night_temp}°, днём {day_temp}°\n"  # Температура
                f"🌬️ Ветер {wind_direction}, {wind_speed} м/с\n"           # Ветер
                f"📊 Атмосферное давление ночью {night_pressure} мм рт.ст., днём {day_pressure} мм рт.ст.\n"  # Давление
                f"🌧️ Вероятность осадков {precipitation}%"                 # Осадки
            )
            weather_data.append(weather_message)
            
            # Сохраняем спарсенные данные
            parsed_data.append({
                "date": date,
                "description": weather_description,
                "night_temp": night_temp,
                "day_temp": day_temp,
                "wind_direction": wind_direction,
                "wind_speed": wind_speed,
                "night_pressure": night_pressure,
                "day_pressure": day_pressure,
                "precipitation": precipitation,
            })
        
        # Добавляем дату обновления и ссылку "Подробнее"
        link = feed.entries[0].get("link", "Ссылка отсутствует")   # Берем ссылку из первого элемента
        weather_data.append(f"🔄 Прогноз обновлён {update_time_formatted}")
        weather_data.append(f"[Подробнее]({link})")
        
        return "\n\n".join(weather_data), update_time_formatted, parsed_data
    except requests.RequestException as e:
        return f"Ошибка при запросе для города {city_name}: {e}", None, []
    except Exception as e:
        return f"Неизвестная ошибка для города {city_name}: {e}", None, []

In [5]:
# Функция для отправки прогноза погоды в Telegram
async def check_and_send_weather(context):
    TELEGRAM_CHAT_ID = "1042080417"  # Замените на ваш chat_id
    
    # Проверяем прогноз для каждого города
    for city_name, city_code in CITIES.items():
        weather, update_time, parsed_data = get_weather_from_rss(city_name, city_code)
        
        # Проверяем, изменилось ли время обновления
        last_update_time = last_weather_data.get(city_code, {}).get("update_time")
        if update_time != last_update_time:
            # Проверяем значительные изменения параметров
            significant_change = False
            last_parsed_data = last_weather_data.get(city_code, {}).get("parsed_data", [])
            for i, current_entry in enumerate(parsed_data):
                if i < len(last_parsed_data):
                    last_entry = last_parsed_data[i]
                    if (
                        current_entry["description"] != last_entry["description"] or
                        abs(current_entry["night_temp"] - last_entry["night_temp"]) > 5 or
                        abs(current_entry["day_temp"] - last_entry["day_temp"]) > 5 or
                        current_entry["wind_direction"] != last_entry["wind_direction"] or
                        abs(current_entry["wind_speed"] - last_entry["wind_speed"]) > 3 or
                        abs(current_entry["night_pressure"] - last_entry["night_pressure"]) > 5 or
                        abs(current_entry["day_pressure"] - last_entry["day_pressure"]) > 5 or
                        abs(current_entry["precipitation"] - last_entry["precipitation"]) > 20
                    ):
                        significant_change = True
                        break
            
            # Если есть значительные изменения или новое время обновления, отправляем уведомление
            if significant_change or not last_update_time:
                await context.bot.send_message(chat_id=TELEGRAM_CHAT_ID, text=weather, parse_mode="Markdown")
                print(f"Прогноз для города {city_name} отправлен.")
            
            # Обновляем состояние
            last_weather_data[city_code] = {"update_time": update_time, "parsed_data": parsed_data}
        else:
            print(f"Прогноз для города {city_name} не изменился.")

In [6]:
# Функция для запуска планировщика
def start_scheduler(application):
    scheduler = AsyncIOScheduler()
    
    # Добавляем задачу на проверку прогнозов каждые 60 минут
    scheduler.add_job(check_and_send_weather, 'interval', minutes=10, args=[application])
    
    # Запускаем планировщик
    scheduler.start()
    print("Планировщик запущен. Прогноз будет проверяться каждые 60 минут.")

In [ ]:
# Основная функция
async def main():
    TELEGRAM_TOKEN = "7991239059:AAFtDMISLWwRbaFiIUZEpJsAqUhqRZS62fo"  # Замените на ваш токен
    
    # Создаем приложение
    application = Application.builder().token(TELEGRAM_TOKEN).build()
    
    # Регистрируем обработчики команд
    application.add_handler(CommandHandler("start", lambda update, context: update.message.reply_text("Бот запущен.")))
    
    # Запускаем планировщик
    start_scheduler(application)
    
    # Запускаем бота
    await application.initialize()
    await application.start()
    await application.updater.start_polling()

if __name__ == "__main__":
    # Убедитесь, что вы не вызываете asyncio.run() вручную
    import nest_asyncio
    nest_asyncio.apply()
    
    # Запускаем основной цикл
    asyncio.run(main())

Планировщик запущен. Прогноз будет проверяться каждые 60 минут.


Прогноз для города Москва (ВДНХ) отправлен.
Прогноз для города Москва (Шереметьево) отправлен.
Прогноз для города Белый отправлен.
Прогноз для города Смоленск отправлен.
Прогноз для города Великие Луки отправлен.
Прогноз для города Торопец отправлен.
Прогноз для города Тверь отправлен.
Прогноз для города Старица отправлен.
Прогноз для города Москва (ВДНХ) не изменился.
Прогноз для города Москва (Шереметьево) не изменился.
Прогноз для города Белый не изменился.
Прогноз для города Смоленск не изменился.
Прогноз для города Великие Луки не изменился.
Прогноз для города Торопец не изменился.
Прогноз для города Тверь не изменился.
Прогноз для города Старица не изменился.
Прогноз для города Москва (ВДНХ) не изменился.
Прогноз для города Москва (Шереметьево) не изменился.
Прогноз для города Белый не изменился.
Прогноз для города Смоленск не изменился.
Прогноз для города Великие Луки не изменился.
Прогноз для города Торопец не изменился.
Прогноз для города Тверь не изменился.
Прогноз для города